# Imports and parameters

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage import binary_dilation, gaussian_filter
from scipy.special import expit

In [ ]:
size = 1024
n = 256
map_seed = 762345

In [ ]:
np.random.seed(map_seed)

# Voronoi diagram

In [ ]:
from minecraft_gen import voronoi, voronoi_map

In [ ]:
points = np.random.randint(0, size, (514, 2))
vor = voronoi(points, size)
vor_map = voronoi_map(vor, size)

In [ ]:
fig, ax = plt.subplots(dpi=150, figsize=(4, 4))
ax.scatter(*points.T, marker=".", color="r")
ax.imshow(vor_map);

# Lloyd's relaxation

In [ ]:
from minecraft_gen import relax

In [ ]:
points = relax(points, size, k=100)
vor = voronoi(points, size)
vor_map = voronoi_map(vor, size)

In [ ]:
fig, ax = plt.subplots(dpi=150, figsize=(4, 4))
ax.scatter(*points.T, marker=".", color="r")
ax.imshow(vor_map);

# Perlin noise / Simplex noise

In [ ]:
from minecraft_gen import blur_boundaries

# Bluring the boundaries

In [ ]:
blurred_vor_map = blur_boundaries(
    vor_map, size=size, map_seed=map_seed, boundary_displacement=8
)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4), dpi=150)
axs[0].imshow(vor_map)
axs[1].imshow(blurred_vor_map);

In [ ]:
vor_map = blurred_vor_map

# Choosing Biomes

In [ ]:
from minecraft_gen import noise_map

## Temperature–Precipitation maps

In [ ]:
temperature_map = noise_map(size, 2, 10, map_seed=map_seed)
precipitation_map = noise_map(size, 2, 20, map_seed=map_seed)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4), dpi=150)

axs[0].imshow(temperature_map, cmap="rainbow")
axs[0].set_title("Temperature Map")

axs[1].imshow(precipitation_map, cmap="YlGnBu")
axs[1].set_title("Precipitation Map");

## Histogram Equalization

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4), dpi=150)

axs[0].hist(
    temperature_map.flatten(),
    bins=64,
    color="blue",
    alpha=0.66,
    label="Precipitation",
)
axs[0].hist(
    precipitation_map.flatten(),
    bins=64,
    color="red",
    alpha=0.66,
    label="Temperature",
)
axs[0].set_xlim(-1, 1)
axs[0].legend()

hist2d = np.histogram2d(
    temperature_map.flatten(),
    precipitation_map.flatten(),
    bins=(512, 512),
    range=((-1, 1), (-1, 1)),
)[0]

hist2d = np.interp(hist2d, (hist2d.min(), hist2d.max()), (0, 1))
hist2d = expit(hist2d / 0.1)

axs[1].imshow(hist2d, cmap="plasma")

axs[1].set_xticks([0, 128, 256, 385, 511])
axs[1].set_xticklabels([-1, -0.5, 0, 0.5, 1])
axs[1].set_yticks([0, 128, 256, 385, 511])
axs[1].set_yticklabels([1, 0.5, 0, -0.5, -1]);

In [ ]:
from minecraft_gen import histeq

In [ ]:
uniform_temperature_map = histeq(temperature_map, alpha=0.33)
uniform_precipitation_map = histeq(precipitation_map, alpha=0.33)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4), dpi=150)

axs[0].hist(
    uniform_temperature_map.flatten(),
    bins=64,
    color="blue",
    alpha=0.66,
    label="Precipitation",
)
axs[0].hist(
    uniform_precipitation_map.flatten(),
    bins=64,
    color="red",
    alpha=0.66,
    label="Temperature",
)
axs[0].set_xlim(-1, 1)
axs[0].legend()

hist2d = np.histogram2d(
    uniform_temperature_map.flatten(),
    uniform_precipitation_map.flatten(),
    bins=(512, 512),
    range=((-1, 1), (-1, 1)),
)[0]

hist2d = np.interp(hist2d, (hist2d.min(), hist2d.max()), (0, 1))
hist2d = expit(hist2d / 0.1)

axs[1].imshow(hist2d, cmap="plasma")

axs[1].set_xticks([0, 128, 256, 385, 511])
axs[1].set_xticklabels([-1, -0.5, 0, 0.5, 1])
axs[1].set_yticks([0, 128, 256, 385, 511])
axs[1].set_yticklabels([1, 0.5, 0, -0.5, -1]);

In [ ]:
temperature_map = uniform_temperature_map
precipitation_map = uniform_precipitation_map

## Averaging Cells

In [ ]:
from minecraft_gen import average_cells, color_cells, fill_cells

In [ ]:
temperature_cells = average_cells(vor_map, temperature_map)
precipitation_cells = average_cells(vor_map, precipitation_map)

temperature_map = fill_cells(vor_map, temperature_cells)
precipitation_map = fill_cells(vor_map, precipitation_cells)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4), dpi=150)

axs[0].imshow(temperature_map, cmap="rainbow")
axs[0].set_title("Temperature")

axs[1].imshow(precipitation_map, cmap="Blues")
axs[1].set_title("Precipitation");

## Quantization

In [ ]:
from minecraft_gen import quantize

quantize_temperature_cells = quantize(temperature_cells, n)
quantize_precipitation_cells = quantize(precipitation_cells, n)

quantize_temperature_map = fill_cells(vor_map, quantize_temperature_cells)
quantize_precipitation_map = fill_cells(vor_map, quantize_precipitation_cells)

In [ ]:
temperature_cells = quantize_temperature_cells
precipitation_cells = quantize_precipitation_cells

temperature_map = quantize_temperature_map
precipitation_map = quantize_precipitation_map

## Temperature–Precipitation graph

In [ ]:
from minecraft_gen import get_biomes

biome_names, biome_colors, biomes = get_biomes("orig/output/TP_map.png")

In [ ]:
fig, ax = plt.subplots(dpi=150, figsize=(4, 4))
ax.imshow(biomes)
ax.set_title("Temperature-Precipitation graph");

## Biome map

In [ ]:
from minecraft_gen import compute_biome_map

biome_map = compute_biome_map(temperature_cells, precipitation_cells, biomes, vor_map)
biome_color_map = color_cells(biome_map, biome_colors)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
ax.imshow(biome_color_map);

# Height Map

In [ ]:
height_map = noise_map(
    size,
    4,
    0,
    octaves=6,
    persistence=0.5,
    lacunarity=2,
    map_seed=map_seed,
)
land_mask = height_map > 0

In [ ]:
fig, ax = plt.subplots(dpi=150, figsize=(5, 5))
ax.imshow(land_mask, cmap="gray");

In [ ]:
sea_color = np.array([12, 14, 255])
land_mask_color = np.repeat(land_mask[:, :, np.newaxis], 3, axis=-1)
masked_biome_color_map = (
    land_mask_color * biome_color_map + (1 - land_mask_color) * sea_color
)

In [ ]:
fig, ax = plt.subplots(dpi=150, figsize=(5, 5))
ax.imshow(masked_biome_color_map);

In [ ]:
# https://github.com/Mehdi-Antoine/NormalMapGenerator

from minecraft_gen import apply_height_map

In [ ]:
biome_height_map, normal_map = apply_height_map(
    masked_biome_color_map,
    height_map,
    height_map,
    land_mask,
)

fig, axs = plt.subplots(1, 2, figsize=(10, 5), dpi=150)

axs[0].imshow(masked_biome_color_map)
axs[0].set_title("Biomes")

axs[1].imshow(np.clip(biome_height_map, 0, 255))
axs[1].set_title("Biomes with normal");

## Height Map Detail

In [ ]:
height_map = noise_map(
    size,
    4,
    0,
    octaves=6,
    persistence=0.5,
    lacunarity=2,
    map_seed=map_seed,
)
smooth_height_map = noise_map(
    size,
    4,
    0,
    octaves=1,
    persistence=0.5,
    lacunarity=2,
    map_seed=map_seed,
)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5), dpi=150)

axs[0].imshow(height_map, cmap="gray")
axs[0].set_title("Height Map")

axs[1].imshow(smooth_height_map, cmap="gray")
axs[1].set_title("Smooth Height Map");

## Height Map Filters

### Bézier Curves

In [ ]:
from minecraft_gen import filter_map

### Filters

In [ ]:
biome_height_maps = [
    # Desert
    filter_map(height_map, smooth_height_map, 0.75, 0.2, 0.95, 0.2, 0.2, 0.5),
    # Savanna
    filter_map(height_map, smooth_height_map, 0.5, 0.1, 0.95, 0.1, 0.1, 0.2),
    # Tropical Woodland
    filter_map(height_map, smooth_height_map, 0.33, 0.33, 0.95, 0.1, 0.1, 0.75),
    # Tundra
    filter_map(height_map, smooth_height_map, 0.5, 1, 0.25, 1, 1, 1),
    # Seasonal Forest
    filter_map(height_map, smooth_height_map, 0.75, 0.5, 0.4, 0.4, 0.33, 0.2),
    # Rainforest
    filter_map(height_map, smooth_height_map, 0.5, 0.25, 0.66, 1, 1, 0.5),
    # Temperate forest
    filter_map(height_map, smooth_height_map, 0.75, 0.5, 0.4, 0.4, 0.33, 0.33),
    # Temperate Rainforest
    filter_map(height_map, smooth_height_map, 0.75, 0.5, 0.4, 0.4, 0.33, 0.33),
    # Boreal
    filter_map(height_map, smooth_height_map, 0.8, 0.1, 0.9, 0.05, 0.05, 0.1),
]

### Biome masks

In [ ]:
biome_count = len(biome_names)
biome_masks = np.zeros((biome_count, size, size))

for i in range(biome_count):
    biome_masks[i, biome_map == i] = 1
    biome_masks[i] = gaussian_filter(biome_masks[i], sigma=16)

# Remove ocean from masks
blurred_land_mask = land_mask
blurred_land_mask = binary_dilation(land_mask, iterations=32).astype(np.float64)
blurred_land_mask = gaussian_filter(blurred_land_mask, sigma=16)

biome_masks = biome_masks * blurred_land_mask

In [ ]:
fig, ax = plt.subplots(dpi=150, figsize=(5, 5))
ax.imshow(biome_masks[6], cmap="gray");

### Applying Filters

In [ ]:
adjusted_height_map = height_map.copy()

for bm, bhm in zip(biome_masks, biome_height_maps):
    adjusted_height_map = (1 - bm) * adjusted_height_map + bm * bhm

In [ ]:
biome_height_map = apply_height_map(
    masked_biome_color_map,
    height_map,
    height_map,
    land_mask,
)
new_biome_height_map = apply_height_map(
    masked_biome_color_map,
    adjusted_height_map,
    adjusted_height_map,
    land_mask,
)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=150)

ax[0].imshow(adjusted_height_map)
ax[0].set_title("Before")

ax[1].imshow(np.clip(new_biome_height_map[0], 0, 255))
ax[1].set_title("After");

# Rivers

## Boundaries

In [ ]:
from minecraft_gen import get_boundary

In [ ]:
biome_bound = get_boundary(biome_map, kernel=5, size=size)
cell_bound = get_boundary(vor_map, kernel=2, size=size)

In [ ]:
river_mask = (
    noise_map(
        size,
        4,
        4353,
        octaves=6,
        persistence=0.5,
        lacunarity=2,
        map_seed=map_seed,
    )
    > 0
)

new_biome_bound = biome_bound * (adjusted_height_map < 0.5) * land_mask
new_cell_bound = cell_bound * (adjusted_height_map < 0.05) * land_mask

rivers = np.logical_or(new_biome_bound, new_cell_bound) * river_mask

In [ ]:
loose_river_mask = binary_dilation(rivers, iterations=8)
rivers_height = gaussian_filter(rivers.astype(np.float64), sigma=2) * loose_river_mask

In [ ]:
adjusted_height_river_map = adjusted_height_map * (1 - rivers_height) - 0.05 * rivers

In [ ]:
river_land_mask = adjusted_height_river_map >= 0
land_mask_color = np.repeat(river_land_mask[:, :, np.newaxis], 3, axis=-1)
rivers_biome_color_map = (
    land_mask_color * biome_color_map + (1 - land_mask_color) * sea_color
)

fig, ax = plt.subplots(dpi=150, figsize=(5, 5))
ax.imshow(rivers_biome_color_map);

In [ ]:
color_map = apply_height_map(
    rivers_biome_color_map,
    adjusted_height_river_map,
    adjusted_height_river_map,
    river_land_mask,
)
plt.imshow(np.clip(color_map[0], 0, 255));

# Trees and Vegetation

In [ ]:
from minecraft_gen import generate_trees, place_trees

In [ ]:
fix, axs = plt.subplots(1, 3, dpi=150, figsize=(10, 3))
densities = [1000, 5000, 25000]
names = ["Low", "Medium", "High"]

for ax, dens, name in zip(axs, densities, names):
    trees = generate_trees(np.random.randint(0, size - 1, (dens, 2)), size=size)

    ax.scatter(*trees.T, s=1)
    ax.set_title(f"{name} Density Trees")
    ax.set_xlim(0, 256)
    ax.set_ylim(0, 256)

In [ ]:
tree_densities = [4000, 1500, 8000, 1000, 10000, 25000, 10000, 20000, 5000]
trees = [
    np.array(
        place_trees(
            generate_trees(np.random.randint(0, size - 1, (dens, 2)), size=size),
            bio_mask,
            river_land_mask=river_land_mask,
            adjusted_height_river_map=adjusted_height_river_map,
            size=size,
        ),
    )
    for dens, bio_mask in zip(tree_densities, biome_masks)
]

In [ ]:
fig, ax = plt.subplots(dpi=150, figsize=(5, 5))
for tree in trees:
    ax.scatter(*tree.T, s=0.15, c="red")

ax.imshow(np.clip(color_map[0], 0, 255));

In [ ]:
def compute_all(*, size=1024, n=256, map_seed=762345, boundary_displacement=8):
    gen = np.random.default_rng(map_seed)
    points_orig = gen.integers(0, size, (514, 2))
    points = relax(points_orig, size, k=100)
    vor = voronoi(points, size)
    vor_map = voronoi_map(vor, size)
    vor_map = blur_boundaries(
        vor_map, size=size, map_seed=map_seed, boundary_displacement=8
    )

    temperature_map = noise_map(size, 2, 10, map_seed=map_seed)
    precipitation_map = noise_map(size, 2, 20, map_seed=map_seed)

    temperature_map = histeq(temperature_map, alpha=0.33)
    precipitation_map = histeq(precipitation_map, alpha=0.33)

    temperature_cells = average_cells(vor_map, temperature_map)
    precipitation_cells = average_cells(vor_map, precipitation_map)

    temperature_map = fill_cells(vor_map, temperature_cells)
    precipitation_map = fill_cells(vor_map, precipitation_cells)

    temperature_cells = quantize(temperature_cells, n)
    precipitation_cells = quantize(precipitation_cells, n)
    temperature_map = fill_cells(vor_map, temperature_cells)
    precipitation_map = fill_cells(vor_map, precipitation_cells)

    biome_names, biome_colors, biomes = get_biomes("orig/output/TP_map.png")

    height_map = noise_map(
        size,
        4,
        0,
        octaves=6,
        persistence=0.5,
        lacunarity=2,
        map_seed=map_seed,
    )
    land_mask = height_map > 0

    sea_color = np.array([12, 14, 255])
    land_mask_color = np.repeat(land_mask[:, :, np.newaxis], 3, axis=-1)
    masked_biome_color_map = (
        land_mask_color * biome_color_map + (1 - land_mask_color) * sea_color
    )

    biome_height_map, normal_map = apply_height_map(
        masked_biome_color_map,
        height_map,
        height_map,
        land_mask,
    )

    height_map = noise_map(
        size,
        4,
        0,
        octaves=6,
        persistence=0.5,
        lacunarity=2,
        map_seed=map_seed,
    )
    smooth_height_map = noise_map(
        size,
        4,
        0,
        octaves=1,
        persistence=0.5,
        lacunarity=2,
        map_seed=map_seed,
    )

    biome_height_maps = [
        # Desert
        filter_map(height_map, smooth_height_map, 0.75, 0.2, 0.95, 0.2, 0.2, 0.5),
        # Savanna
        filter_map(height_map, smooth_height_map, 0.5, 0.1, 0.95, 0.1, 0.1, 0.2),
        # Tropical Woodland
        filter_map(height_map, smooth_height_map, 0.33, 0.33, 0.95, 0.1, 0.1, 0.75),
        # Tundra
        filter_map(height_map, smooth_height_map, 0.5, 1, 0.25, 1, 1, 1),
        # Seasonal Forest
        filter_map(height_map, smooth_height_map, 0.75, 0.5, 0.4, 0.4, 0.33, 0.2),
        # Rainforest
        filter_map(height_map, smooth_height_map, 0.5, 0.25, 0.66, 1, 1, 0.5),
        # Temperate forest
        filter_map(height_map, smooth_height_map, 0.75, 0.5, 0.4, 0.4, 0.33, 0.33),
        # Temperate Rainforest
        filter_map(height_map, smooth_height_map, 0.75, 0.5, 0.4, 0.4, 0.33, 0.33),
        # Boreal
        filter_map(height_map, smooth_height_map, 0.8, 0.1, 0.9, 0.05, 0.05, 0.1),
    ]

    biome_count = len(biome_names)
    biome_masks = np.zeros((biome_count, size, size))

    for i in range(biome_count):
        biome_masks[i, biome_map == i] = 1
        biome_masks[i] = gaussian_filter(biome_masks[i], sigma=16)

    # Remove ocean from masks
    blurred_land_mask = land_mask
    blurred_land_mask = binary_dilation(land_mask, iterations=32).astype(np.float64)
    blurred_land_mask = gaussian_filter(blurred_land_mask, sigma=16)

    biome_masks = biome_masks * blurred_land_mask

    adjusted_height_map = height_map.copy()

    for bm, bhm in zip(biome_masks, biome_height_maps):
        adjusted_height_map = (1 - bm) * adjusted_height_map + bm * bhm

    biome_height_map = apply_height_map(
        masked_biome_color_map,
        height_map,
        height_map,
        land_mask,
    )
    new_biome_height_map = apply_height_map(
        masked_biome_color_map,
        adjusted_height_map,
        adjusted_height_map,
        land_mask,
    )

    biome_bound = get_boundary(biome_map, kernel=5, size=size)
    cell_bound = get_boundary(vor_map, kernel=2, size=size)

    river_mask = (
        noise_map(
            size,
            4,
            4353,
            octaves=6,
            persistence=0.5,
            lacunarity=2,
            map_seed=map_seed,
        )
        > 0
    )

    new_biome_bound = biome_bound * (adjusted_height_map < 0.5) * land_mask
    new_cell_bound = cell_bound * (adjusted_height_map < 0.05) * land_mask

    rivers = np.logical_or(new_biome_bound, new_cell_bound) * river_mask

    loose_river_mask = binary_dilation(rivers, iterations=8)
    rivers_height = (
        gaussian_filter(rivers.astype(np.float64), sigma=2) * loose_river_mask
    )

    adjusted_height_river_map = (
        adjusted_height_map * (1 - rivers_height) - 0.05 * rivers
    )

    river_land_mask = adjusted_height_river_map >= 0
    land_mask_color = np.repeat(river_land_mask[:, :, np.newaxis], 3, axis=-1)
    rivers_biome_color_map = (
        land_mask_color * biome_color_map + (1 - land_mask_color) * sea_color
    )

    color_map = apply_height_map(
        rivers_biome_color_map,
        adjusted_height_river_map,
        adjusted_height_river_map,
        river_land_mask,
    )

    tree_densities = [4000, 1500, 8000, 1000, 10000, 25000, 10000, 20000, 5000]
    trees = [
        np.array(
            place_trees(
                generate_trees(np.random.randint(0, size - 1, (dens, 2)), size=size),
                bio_mask,
                river_land_mask=river_land_mask,
                adjusted_height_river_map=adjusted_height_river_map,
                size=size,
            ),
        )
        for dens, bio_mask in zip(tree_densities, biome_masks)
    ]

    return color_map, trees

In [ ]:
color_map, trees = compute_all()
fig, ax = plt.subplots(dpi=150, figsize=(5, 5))
for tree in trees:
    ax.scatter(*tree.T, s=0.15, c="red")

ax.imshow(np.clip(color_map[0], 0, 255));